In [1]:
# import required packages-

import pandas as pd
import datetime
import matplotlib.pyplot as plt
import matplotlib.font_manager as fnt
from mplsoccer import VerticalPitch

In [2]:
# Set input paramaters such as titles, colours and squad file.abs

team_name = "Aberdeen"

season_start = 2025

squadfile = 'aberdeen.csv'

colours = {
    "cl_background":"#000000",
    "cl_foreground":"#FFFFFF",
    "cl_secondary":"#999999",
    "cl_alt_1":"#d61f34",
    "cl_alt_2":"#3A86FF",
    "cl_alt_3":"#FFBE0B",
    "cl_alt_4":"#3EC300",
    "cl_alt_5":"#595959"}


title = "Squad age distribution for " + team_name + " season starting " + str(season_start) 
subtitle = "On basis of current date: " + datetime.datetime.now().strftime("%d/%m/%Y") 

# Set Fonts
title_font = fnt.FontProperties(family="P052")
body_font = fnt.FontProperties(family="URW Gothic")

In [48]:
def calculate_age(born, season_start):
    start_date = datetime.datetime(season_start, 8,1)
    cutpoint = ((start_date.month < born.dt.month) & (start_date.day < born.dt.day))
    age = start_date.year - born.dt.year 
    age[cutpoint] = age -1
    return age

In [49]:
# read in and dispaly header of csv files to check.
squad = pd.read_csv(squadfile)
squad["Date of Birth"] = pd.to_datetime(squad["Date of Birth"])
squad["Age"] = calculate_age(squad["Date of Birth"],season_start)
squad.head()


,First Name,Last Name,Date of Birth,Position,Contract Expiry,Status,Age
0,Dimitar,Mitov,1997-01-22,GK,2027,N,28
1,Kristers,Tobers,2000-12-13,RCB,2028,N,24
2,Mats,Knoester,1998-11-19,LCB,2027,N,26
3,Gavin,Molloy,2001-10-19,LCB,2027,N,23
4,Jack,Milne,2003-02-10,RCB,2029,N,22
